In [1]:
import pandas as pd
import numpy as np
from src import master_calendar
from src import session_identification


#### Simplification: droping & renaming columns

In [2]:
df_raw = pd.read_csv("data/raw/concept2-season-2020.csv")

# columns to drop (empty, with duplicated values or identical constants)
col_to_drop = [
    "Stroke Count",  # empty
    "Avg Heart Rate",  # empty
    "Comments",  # empty
    "Weight",  # constant ("Lwt")
    "Type",  # constant ("Indoor Rower")
    "Ranked",  # constant ("No")
    "Work Time (Formatted)",  # string duplicate (of float "Work Time (seconds)")
    "Rest Time (Formatted)",  #  string duplicate (of float "Rest Time (seconds)")
    "Age",  # constant ("26" or "27", negligeable difference)
    "Cal/Hour",  # duplicate (similar to power in watts per units of time)
]
df = df_raw.drop(col_to_drop, axis="columns")

In [3]:
# renaming based on preferences
dict_naming = {
    "ID": "session_id",
    "Description": "session_type",
    "Date": "session_timestamp",
    "Work Time (Seconds)": "work_in_s",
    "Rest Time (Seconds)": "rest_in_s",
    "Work Distance": "work_in_m",
    "Rest Distance": "rest_in_m",
    "Stroke Rate/Cadence": "spm",
    "Pace": "time_split",
    "Total Cal": "cal_burned",
    "Drag Factor": "drag_factor",
    
    "Avg Watts": "power_in_avg_w",
}
df.rename(columns=dict_naming, inplace=True)

df["session_date"] = pd.to_datetime(df["session_timestamp"]).dt.date
df["session_date"] = pd.to_datetime(df["session_date"])
df["session_timestamp"] = pd.to_datetime(df["session_timestamp"])
df["time_split"] = pd.to_datetime(df["time_split"]).dt.time

df.head(3)

,session_id,session_timestamp,session_type,work_in_s,rest_in_s,work_in_m,rest_in_m,spm,time_split,power_in_avg_w,cal_burned,drag_factor,session_date
0,42587184,2019-12-10 18:37:00,v1000m/1:00r...10 row,752.6,600.0,3200,584.0,27,01:57:30,215,219,116,2019-12-10
1,42587183,2019-12-09 08:43:00,5000m row,1425.7,NaN,5000,NaN,23,02:22:30,121,296,111,2019-12-09
2,42587182,2019-12-07 14:24:00,5000m row,1404.7,NaN,5000,NaN,22,02:20:24,126,308,115,2019-12-07


### Description of remaining columns:
| Column            | Dtype | Description                                                                                                                         |
| ----------------- | ----- | ----------------------------------------------------------------------------------------------------------------------------------- |
|*session_id*       |int    |unique value for the session                                                                                                         |
|*session_timestamp*|dt     |timestamp (date + time) of the session                                                                                               |
|*session_type*     |object |type of rowing session such as distance (2000m, 5000m...), time (30:00, 24:34 ...) or a mix as interval (v1000m/1:00r...10 row, etc) |
|**work_in_s**      |float  |length in seconds of effort during the session, never equals to 0 during session                                                     |
|*rest_in_s*        |float  |length in seconds of rest during the session ( = low or no watts of power), *equal to 0 if no rest period in session*                |
|**work_in_m**      |int    |distance in meters of effort during the session, never equals to 0 during session                                                    |
|*rest_in_m*        |float  |distance in meters of rest during the session ( = low or no watts of power), *equal to 0 if no rest period in session*               |
|**spm**            |int    |average stroke per minute for the session (*proxy for **speed of movement**, not of power*)                                          |
|**time_split**     |object |average time taken to perform 500m (*proxy for **power**, not of speed of movement*)                                                 |
|*power_in_avg_w*   |int    |average power output delivered accross the session at each stroke                                                                    |
|*cal_burned*       |int    |calories theoritically burned during the session, sum of power delivered in the session                                              |
|*drag_factor*      |int    |opposed air resistance during the session, water is between 115 and 130                                                              |
|*session_date*     |dt     |calendar date of the session                                                                                                         |

#### Master calendar: creating & merging dates

In [4]:
# get first and last rowing dates and generate a master calendar in the intervals
d_M = np.max(df["session_timestamp"])
d_m = np.min(df["session_timestamp"])

print(f"beginning: {d_m}\nend: {d_M}")

df_calendar = master_calendar.create_calendar(d_m, d_M)


beginning: 2019-05-04 20:29:00
end: 2019-12-10 18:37:00


In [5]:
# merge master calendar with training sessions information for temporal aggregation after
df_cal = pd.merge(
    df,
    df_calendar,
    how="right",
    left_on=df["session_date"],
    right_on=df_calendar["mc_date"],
)
# fill NaN values from sessions without rest with 0 (rest but equal to 0)
dict_nan = {
    "rest_in_s": 0,
    "rest_in_m": 0,
    "cal_burned": 0,
    "session_type": "None",
    "session_id": "None",
    "session_timestamp": "None",
    "work_in_m": 0,
    "work_in_s": 0,
    "spm": 0,
    "power_in_avg_w": 0,
    "drag_factor": 0,
    "time_split": 0,
}

df_cal.fillna(value=dict_nan, inplace=True)
df_cal.drop(
    ["calendar", "session_date", "key_0"], axis="columns", inplace=True
)  # duplicated information


#### Training sessions: identification & grouping

In [6]:
df_cal["training_day"] = (df_cal["session_type"] != "None").astype(
    bool
)  # simple boolean filter for training days
df_cal["multi_training"] = df_cal.duplicated(
    subset="mc_date", keep=False
)  # tag for multiple sessions the same day


In [7]:
# If not categorized, labelled as other
df_cal[["session_type_group","session_type_class"]] = pd.DataFrame(df_cal["session_type"].apply(session_identification.categorizeSession).to_list())

In [8]:
# Store as pickle for the visualization notebook
df_cal.to_pickle("data/interim/c2.pkl")
df_cal.to_json("data/interim/c2.json", orient="index")
df_cal.head(3)

,session_id,session_timestamp,session_type,work_in_s,rest_in_s,work_in_m,rest_in_m,spm,time_split,power_in_avg_w,...,mc_week,mc_month,mc_quarter,mc_year_half,mc_year,mc_yearMonth,training_day,multi_training,session_type_group,session_type_class
0,38440889.0,2019-05-04 20:29:00,1x500m/1:00r row,113.3,60.0,500.0,14.0,31.0,01:53:18,241.0,...,18,05,2,1,2019,2019_05,True,False,Distance Interval Training,Interval Training
1,38440890.0,2019-05-05 10:24:00,5:05 row,305.1,0.0,1034.0,0.0,23.0,02:27:30,109.0,...,18,05,2,1,2019,2019_05,True,False,5:05 row,Timed Row
2,None,None,None,0.0,0.0,0.0,0.0,0.0,0,0.0,...,19,05,2,1,2019,2019_05,False,False,None,None
